# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [1]:
import ee
from utils import *
from config import *
import pandas as pd
from datetime import datetime

In [2]:
service_account = 'jupyter-gee-project@ee-avs-dse.iam.gserviceaccount.com'
key_path = '../service_account_key.json'

credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials)

print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Class Definitions

In [3]:
def main(protected_area_name, year):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    stats_ops = StatsOperations()
    viz = Visualization()
    feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)
    exporter = ExportResults()

    # Load and process protected area geometry
    pa = load_local_data(protected_area_name)
    pa_geometry = pa.geometry()
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)
    aoi_with_biome = geo_ops.get_biome(aoi)

    # Process imagery and add indices
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    image = img_ops.add_indices_to_image(composite)

    # Process features and collect statistics
    feature_info = feature_processor.collect_feature_info(pa, aoi_with_biome)
    computed_stats = feature_processor.process_all_bands(image, pa_geometry)
    all_stats = feature_processor.compile_statistics(feature_info, computed_stats, year)
    
    # Save results and create visualization
    df, _ = exporter.save_statistics_to_csv(all_stats, protected_area_name, year)
    first_band_stats = computed_stats[0]  # Use the last processed band for visualization
    Map = viz.create_map(pa_geometry, first_band_stats['gradient'], first_band_stats['boundary_pixels'])
    
    return Map, df

In [4]:
Map, results_df = main("916", 2010)
display(results_df.head())
Map

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_0/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_0_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)


Results saved to results_916_2010_20250612_011259.csv


,WDPA_PID,ORIG_NAME,BIOME_NAME,GIS_AREA,band_name,year,boundary_x_mean,boundary_x_stdDev,boundary_x_sum,buffer_x_mean,buffer_x_stdDev,buffer_x_sum
0,916,Serengeti National Park,"Tropical & Subtropical Grasslands, Savannas & ...",13123.0503012899,sur_refl_b01,2010,0.090834,0.072125,261.699499,0.090834,0.072125,261.699499
1,916,Serengeti National Park,"Tropical & Subtropical Grasslands, Savannas & ...",13123.0503012899,sur_refl_b02,2010,0.159655,0.113046,459.978873,0.159655,0.113046,459.978873
2,916,Serengeti National Park,"Tropical & Subtropical Grasslands, Savannas & ...",13123.0503012899,sur_refl_b03,2010,0.045173,0.037031,130.148115,0.045173,0.037031,130.148115
3,916,Serengeti National Park,"Tropical & Subtropical Grasslands, Savannas & ...",13123.0503012899,EVI,2010,0.000110,0.000084,0.318072,0.000110,0.000084,0.318072
4,916,Serengeti National Park,"Tropical & Subtropical Grasslands, Savannas & ...",13123.0503012899,NDVI,2010,0.000042,0.000032,0.120636,0.000042,0.000032,0.120636


Map(center=[-2.33326594708425, 34.78445454288309], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
try:
    # Load and verify geometry
    pa = load_local_data("367731")
    pa2 = load_protected_area("367731")
    print("1. Successfully loaded protected area")
    
    # Create map with debug prints
    import geemap
    Map = geemap.Map()
    print("2. Created base map")
    
    # Add basemap and geometry with error checking
    try:
        Map.add_basemap('HYBRID')
        Map.addLayer(
            ee.FeatureCollection([pa]), 
            {'color': 'red', 'fillColor': '#ff000033', 'width': 2},
            'Protected Area'
        )
        Map.addLayer(
            ee.FeatureCollection([pa2]), 
            {'color': 'blue', 'fillColor': '#ff000033', 'width': 2},
            'Protected Area2'
        )
        bounds = pa.geometry().bounds()
        Map.centerObject(bounds, zoom=8)
        display(Map)
        print("3. Map displayed successfully")
        
    except Exception as e:
        print(f"Map visualization error: {e}")
        
except Exception as e:
    print(f"Error: {e}")
    if 'pa' in locals():
        print("\nDebug info:")
        print(f"Feature type: {type(pa)}")
        print(f"Properties: {pa.propertyNames().getInfo()}")


1. Successfully loaded protected area
2. Created base map


Map(center=[36.46793434725051, -117.13877499999978], controls=(WidgetControl(options=['position', 'transparent…

3. Map displayed successfully


In [6]:
try:
    # Read shapefile 
    shp_path = '../data/global_wdpa_June2021/Global_wdpa_wInfo_June2021.shp'
    gdf = gpd.read_file(shp_path)
    
    # Get unique WDPA_PIDs and count
    unique_pids = gdf['WDPA_PID'].unique()
    print(f"Total number of unique WDPA_PIDs: {len(unique_pids)}")
    
    # Show first few PIDs and their park names
    print("\nSample of protected areas:")
    sample_df = gdf[['WDPA_PID', 'NAME']].head()
    print(sample_df.to_string(index=False))
    
except Exception as e:
    print(f"Error reading shapefile: {e}")

Total number of unique WDPA_PIDs: 6952

Sample of protected areas:
WDPA_PID                   NAME
       3 Laguna de los Pozuelos
       6          Los Glaciares
       7                  Lanín
       8            Los Alerces
      10              Calilegua


-add write out for each step, with identifier for each park
-ray to run in parallel in python
-use glance to check usage